In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from smallvggnet import SmallVGGNet
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [3]:
# ArgParser Things
dataset = "animals"
plot = "output/train_conv_val.png"
modelPath = "output/train_conv_nn.model"
label_bin = "output/train_conv_nn_lb.pickle"

In [5]:
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(dataset)))
random.seed(42)
random.shuffle(imagePaths)
# loop over the input images
for imagePath in imagePaths:
	# load the image, resize it to 64x64 pixels (the required input
	# spatial dimensions of SmallVGGNet), and store the image in the
	# data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (64, 64))
	data.append(image)
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

[INFO] loading images...


In [6]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [7]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")
# initialize our VGG-like Convolutional Neural Network
model = SmallVGGNet.build(width=64, height=64, depth=3,
	classes=len(lb.classes_))

In [8]:
# initialize our initial learning rate, # of epochs to train for,
# and batch size
INIT_LR = 0.01
EPOCHS = 75
BS = 32
# initialize the model and optimizer (you'll want to use
# binary_crossentropy for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the network
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)

[INFO] training network...
Train for 70 steps, validate on 750 samples
Epoch 1/75
70/70 [==============================] - 62s 889ms/step - loss: 1.3870 - accuracy: 0.5203 - val_loss: 1.6473 - val_accuracy: 0.3480
Epoch 2/75
70/70 [==============================] - 52s 746ms/step - loss: 1.0260 - accuracy: 0.5834 - val_loss: 2.2945 - val_accuracy: 0.3147
Epoch 3/75
70/70 [==============================] - 52s 740ms/step - loss: 0.9079 - accuracy: 0.6064 - val_loss: 1.7899 - val_accuracy: 0.3173
Epoch 4/75
70/70 [==============================] - 59s 849ms/step - loss: 0.8261 - accuracy: 0.6217 - val_loss: 1.0656 - val_accuracy: 0.4440
Epoch 5/75
70/70 [==============================] - 64s 914ms/step - loss: 0.7618 - accuracy: 0.6411 - val_loss: 1.3592 - val_accuracy: 0.4027
Epoch 6/75
70/70 [==============================] - 56s 802ms/step - loss: 0.7443 - accuracy: 0.6434 - val_loss: 0.9807 - val_accuracy: 0.5213
Epoch 7/75
70/70 [==============================] - 61s 869ms/step - lo

70/70 [==============================] - 54s 773ms/step - loss: 0.4270 - accuracy: 0.8034 - val_loss: 0.6630 - val_accuracy: 0.7627
Epoch 58/75
70/70 [==============================] - 54s 772ms/step - loss: 0.4545 - accuracy: 0.7958 - val_loss: 0.6238 - val_accuracy: 0.7600
Epoch 59/75
70/70 [==============================] - 55s 792ms/step - loss: 0.4210 - accuracy: 0.8034 - val_loss: 0.5888 - val_accuracy: 0.7667
Epoch 60/75
70/70 [==============================] - 60s 860ms/step - loss: 0.4215 - accuracy: 0.8133 - val_loss: 0.6553 - val_accuracy: 0.7613
Epoch 61/75
70/70 [==============================] - 54s 773ms/step - loss: 0.4115 - accuracy: 0.8161 - val_loss: 1.2626 - val_accuracy: 0.6173
Epoch 62/75
70/70 [==============================] - 54s 777ms/step - loss: 0.4188 - accuracy: 0.8097 - val_loss: 0.6150 - val_accuracy: 0.7547
Epoch 63/75
70/70 [==============================] - 53s 761ms/step - loss: 0.4048 - accuracy: 0.8161 - val_loss: 0.6220 - val_accuracy: 0.7613
Epoc

In [9]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(plot)
# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save(modelPath, save_format="h5")
f = open(label_bin, "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] evaluating network...
              precision    recall  f1-score   support

        cats       0.69      0.69      0.69       236
        dogs       0.72      0.46      0.56       236
       panda       0.75      0.97      0.85       278

    accuracy                           0.72       750
   macro avg       0.72      0.71      0.70       750
weighted avg       0.72      0.72      0.71       750

[INFO] serializing network and label binarizer...
